In [1]:
from tensorflow.python.framework.ops import disable_eager_execution
from sklearn.model_selection import train_test_split
disable_eager_execution()
import nltk
import re
import pandas as pd
import numpy as np
import math

In [2]:
df = pd.read_csv('D:/tesis/scisumm/scisumm.csv')
# df.head()

In [3]:
# tokenize per sent
from nltk.tokenize import sent_tokenize
sent_for_print = [sent_tokenize(s.lower()) for s in df['text']]

In [4]:
X = df['text']
y = df['summary']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, train_size = .75)

In [5]:
X_train = X_train.str.lower()
X_test  = X_test.str.lower()

In [6]:
tokenizer = nltk.RegexpTokenizer(r"\w+")
train   = [tokenizer.tokenize(s.lower()) for s in X_train]
test    = [tokenizer.tokenize(s.lower()) for s in X_test]

In [7]:
# Stopword Removal
from nltk.corpus import stopwords
nltk.download('stopwords')
listStopword = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
# Menghapus kata-kata di stopwords
train_stopwords = []
for sent in train:
  filter = [s for s in sent if s not in listStopword]
  train_stopwords.append(filter)
# train_stopwords

test_stopwords = []
for sent in test:
  filter = [s for s in sent if s not in listStopword]
  test_stopwords.append(filter)
# test_stopwords

# output merupakan sisa kata yg sudah dihapus

In [9]:
train   = [' '.join(t) for t in train_stopwords]
test    = [' '.join(t) for t in test_stopwords]

In [10]:
# importing modules
from nltk.stem import PorterStemmer
ps = PorterStemmer()

train   = [ps.stem(w) for w in train]
test    = [ps.stem(w) for w in test]

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
vec = TfidfVectorizer()

train   = vec.fit_transform(train)
test    = vec.transform(test)

train   = train.toarray()
test    = test.toarray()

In [12]:
import tensorflow as tf
from keras.layers import Input, Dense, Lambda
from keras import backend as K
from keras import Model

# Setup the network parameters:
original_dim = train.shape[1]
input_shape = (original_dim, )
# intermediate_dim = 200
batch_size  = 16
latent_dim  = 250
epochs      = 10

# Map inputs to the latent distribution parameters:
# VAE model = encoder + decoder
# build encoder model
inputs      = Input(shape=input_shape, name='encoder_input')
# x           = Dense(intermediate_dim, activation='relu')(inputs)
x           = Dense(1000, activation='relu')(inputs)
x           = Dense(500, activation='relu')(x)
z_mean      = Dense(latent_dim, name='z_mean')(x)
z_log_var   = Dense(latent_dim, name='z_log_var')(x)

# Use those parameters to sample new points from the latent space:
# reparameterization trick
# instead of sampling from Q(z|X), sample epsilon = N(0,I)
# z = z_mean + sqrt(var) * epsilon
def sampling(args):
    """Reparameterization trick by sampling from an isotropic unit Gaussian.
    # Arguments
        args (tensor): mean and log of variance of Q(z|X)
    # Returns
        z (tensor): sampled latent vector
    """

    z_mean, z_log_var = args
    batch   = K.shape(z_mean)[0]
    dim     = K.int_shape(z_mean)[1]
    # by default, random_normal has mean = 0 and std = 1.0
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

# use reparameterization trick to push the sampling out as input
# note that "output_shape" isn't necessary with the TensorFlow backend
z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

# Instantiate the encoder model:
encoder = Model(inputs, z_mean, name='encoder')
encoder.summary()

Model: "encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_input (InputLayer)  [(None, 60704)]           0         
                                                                 
 dense (Dense)               (None, 1000)              60705000  
                                                                 
 dense_1 (Dense)             (None, 500)               500500    
                                                                 
 z_mean (Dense)              (None, 250)               125250    
                                                                 
Total params: 61,330,750
Trainable params: 61,330,750
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Build the decoder model:
latent_inputs   = Input(shape=(latent_dim,), name='z_sampling')
# x               = Dense(intermediate_dim, activation='relu')(latent_inputs)
x               = Dense(500, activation='relu')(latent_inputs)
x               = Dense(1000, activation='relu')(x)
outputs         = Dense(original_dim, activation='sigmoid')(x)

# Instantiate the decoder model:
decoder = Model(latent_inputs, outputs, name='decoder')
decoder.summary()

Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 z_sampling (InputLayer)     [(None, 250)]             0         
                                                                 
 dense_2 (Dense)             (None, 500)               125500    
                                                                 
 dense_3 (Dense)             (None, 1000)              501000    
                                                                 
 dense_4 (Dense)             (None, 60704)             60764704  
                                                                 
Total params: 61,391,204
Trainable params: 61,391,204
Non-trainable params: 0
_________________________________________________________________


In [14]:
# Instantiate the VAE model:
outputs = decoder(encoder(inputs))
vae     = Model(inputs, outputs, name='vae_mlp')
vae.summary()

# As in the Keras tutorial, we define a custom loss function:
loss_object = tf.keras.losses.BinaryCrossentropy()
def vae_loss(x, x_decoded_mean):
    xent_loss   = loss_object(x, x_decoded_mean)
    kl_loss     = - 0.5 * K.mean(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
    return xent_loss + kl_loss

Model: "vae_mlp"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_input (InputLayer)  [(None, 60704)]           0         
                                                                 
 encoder (Functional)        (None, 250)               61330750  
                                                                 
 decoder (Functional)        (None, 60704)             61391204  
                                                                 
Total params: 122,721,954
Trainable params: 122,721,954
Non-trainable params: 0
_________________________________________________________________


In [15]:
# We compile the model:
vae.compile(optimizer='rmsprop', loss=vae_loss)

# Finally, we train the model:

results = vae.fit(train, train,
        shuffle=True,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(test, test)
        )

Train on 756 samples, validate on 253 samples
Epoch 1/10
756/756 [==============================] - ETA: 0s - loss: 0.0767

c:\Users\LENOVO\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training_v1.py:2335: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


756/756 [==============================] - 73s 97ms/sample - loss: 0.0767 - val_loss: 0.0033
Epoch 2/10
756/756 [==============================] - 61s 80ms/sample - loss: 0.0028 - val_loss: 0.0021
Epoch 3/10
756/756 [==============================] - 60s 79ms/sample - loss: 0.0022 - val_loss: 0.0020
Epoch 4/10
756/756 [==============================] - 60s 79ms/sample - loss: 0.0021 - val_loss: 0.0019
Epoch 5/10
756/756 [==============================] - 60s 79ms/sample - loss: 0.0020 - val_loss: 0.0020
Epoch 6/10
756/756 [==============================] - 60s 79ms/sample - loss: 0.0020 - val_loss: 0.0019
Epoch 7/10
756/756 [==============================] - 60s 79ms/sample - loss: 0.0020 - val_loss: 0.0019
Epoch 8/10
756/756 [==============================] - 64s 85ms/sample - loss: 0.0020 - val_loss: 0.0019
Epoch 9/10
756/756 [==============================] - 61s 81ms/sample - loss: 0.0020 - val_loss: 0.0019
Epoch 10/10
756/756 [==============================] - 61s 81ms/sample - lo

In [16]:
X_encoded = encoder.predict(train)
reconstruction = vae.predict(train)

print("\n")
print("Variational Autoencoder Encoder Reduced Scissum Dataset Size: : ", X_encoded.shape)
print("\n")
print("Variational Autoencoder Scissum Dataset Size: : ", reconstruction.shape)
print("\n")

c:\Users\LENOVO\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,




Variational Autoencoder Encoder Reduced Scissum Dataset Size: :  (756, 250)


Variational Autoencoder Scissum Dataset Size: :  (756, 60704)




In [17]:
def cosine_similarity(sent_1,sent_2):
  s1_dot_s2=np.sum(np.multiply(sent_1,sent_2))
  magnitude_of_s1=math.sqrt(np.sum(np.multiply(sent_1,sent_1)))
  magnitude_of_s2=math.sqrt(np.sum(np.multiply(sent_2,sent_2)))
  return s1_dot_s2/(magnitude_of_s1*magnitude_of_s2)

In [18]:
def cosine_matrix(reconstruction):
  cosine_similarity_matrix=np.zeros(shape=(len(reconstruction),len(reconstruction)))
  for i in range(len(reconstruction)):
    for j in range(len(reconstruction)):
      cosine_similarity_matrix[i][j]=cosine_similarity(reconstruction[i],reconstruction[j])
  #normalize the matrix
  for idx in range(len(cosine_similarity_matrix)):
    cosine_similarity_matrix[idx] /= cosine_similarity_matrix[idx].sum()
    
  return cosine_similarity_matrix

cosine_similarity_matrix=cosine_matrix(reconstruction)
# print(cosine_similarity_matrix)

In [19]:
def text_rank(cosine_matrix,eps=0.0001,d=0.85):
  teleportation_factor=np.ones(len(reconstruction))/len(reconstruction)
  i=0
  while i<100:
    new_p=np.dot(cosine_similarity_matrix.T,teleportation_factor.T)
    if i<=100:
      return new_p
    i=i+1
    teleportation_factor = new_p
  
  
result_rank=text_rank(cosine_similarity_matrix)
print(len(reconstruction))
rank_with_sentence={}
for i in range(len(sent_for_print)):
  i = 0
  if result_rank[i] not in set(rank_with_sentence.keys()): # error
    rank_with_sentence[result_rank[i]]=[sent_for_print[i]]
  else:
    rank_with_sentence[result_rank[i]].append(sent_for_print[i])

#sentences with their rank
for key in sorted(rank_with_sentence.keys(),reverse=True)[:int(len(rank_with_sentence.keys())/4)]:
  for k in rank_with_sentence[key]:
    # print(key,k)
    k[0].upper()
    print(k) #### why??? kok gabisa di print
    print("helaaaw?")

756
